In [1]:
%config Completer.use_jedi = False
%run spark-instance.ipynb

# system imports
import os
import time 
from datetime import datetime

# installed pkg imports
import numpy as np
import pandas as pd
import ipaddress

pd.set_option('display.max_rows', 300)

# spark imports
import pyspark.sql.functions as psf
import pyspark.sql.types as pst
from pyspark.sql.functions import * 
from pyspark.sql.functions import monotonically_increasing_id

# Other imports 
from pyspark.sql.functions import col, udf 
from pyspark.sql.types import BooleanType
import matplotlib.pyplot as plt
from pyspark_dist_explore import hist
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default='iframe'

# local python files imports

# you have access to the following dates:
#year=2023/month=08/day=22 - 1
#year=2023/month=09/day=19 - 2
#year=2023/month=10/day=24 - 3
#year=2023/month=11/day=21 - 4

#Change the value of date between 1-8. 

date = 2

match date:
    case 1:
        ts = datetime(2023, 8, 22)
    case 2:
        ts = datetime(2023, 9, 19)
    case 3:
        ts = datetime(2023, 10, 24)
    case 4:
        ts = datetime(2023, 11, 21)
    
CENSYS_BASE_PATH_FMT = "../dataset/censys/dataset=universal-internet-dataset-v2-ipv4/format=parquet/"
CENSYS_PATH_FMT = os.path.join(CENSYS_BASE_PATH_FMT, "year={year}/month={month:02d}/day={day:02d}")

# Useful methods applied during research.

def is_list_all_none(lst):
    if isinstance(lst, list):
        return all(x is None for x in lst)
    return False

def extract_value(lst):
    if isinstance(lst, list):
        for item in lst:
            if item is not None:
                return item
    return None

def extract_list_with_strings(lst_of_lsts):
    if isinstance(lst_of_lsts, list):
        for sublist in lst_of_lsts:
            if isinstance(sublist, list) and any(isinstance(item, str) for item in sublist):
                return sublist
    return None
    
def count_versions(sublists, version):
    if sublists is not None:
        count = 0
        for sublist in sublists:
            if sublist is not None:
                count += sublist.count(version)
        return count
    return 0


def categorize_list(lst):
    if set(lst) == {'v1', 'v2c', 'v3'}:
        return 'list_v1_v2c_v3'
    elif set(lst) == {'v1'}:
        return 'list_v1'
    elif set(lst) == {'v2c'}:
        return 'list_v2c'
    elif set(lst) == {'v3'}:
        return 'list_v3'
    elif set(lst) == {'v1', 'v2c'}:
        return 'list_v1_v2c'
    elif set(lst) == {'v1', 'v3'}:
        return 'list_v1_v3'
    elif set(lst) == {'v2c', 'v3'}:
        return 'list_v2c_v3'
    return 'other'

def remove_nones(lst):
    if isinstance(lst, list):
        return [item for item in lst if item is not None]
    return lst

censys_path = CENSYS_PATH_FMT.format(year=ts.year, month=ts.month, day=ts.day)
censys_df = spark.read.option("basePath", CENSYS_BASE_PATH_FMT).parquet(censys_path)

censys_df = censys_df.drop("host_identifier", "snapshot_date", "ipv6_int", "year", "month", "day")

info_df = censys_df.filter((array_contains(col("ports_list"), 161) | array_contains(col("ports_list"), 162)))

main_df = info_df.select(
    "ipv4_int",
    "location.country", 
    "location.city",
    "location.continent",
    "operating_system.vendor", 
    "operating_system.version", 
    "autonomous_system.asn",
    "autonomous_system.country_code",)

main_df = main_df.withColumn("id", monotonically_increasing_id())

working_df = main_df.toPandas()

AS_Country_Vendor_df = working_df[['country_code', 'country', 'vendor', 'continent']]
Vendor_Version_Country_df = working_df[['vendor', 'version', 'country']]
Vendor_Country_df = working_df[['vendor', 'country', 'continent']]

SparkConf created
Started SparkSession
Spark version 3.5.0


In [2]:
Engine_ID = working_df

In [16]:
display(info_df.count())

1495696

In [4]:
country_code_mapping = {
    'AD': 'AND',
    'AE': 'ARE',
    'AF': 'AFG',
    'AG': 'ATG',
    'AI': 'AIA',
    'AL': 'ALB',
    'AM': 'ARM',
    'AO': 'AGO',
    'AQ': 'ATA',
    'AR': 'ARG',
    'AS': 'ASM',
    'AT': 'AUT',
    'AU': 'AUS',
    'AW': 'ABW',
    'AX': 'ALA',
    'AZ': 'AZE',
    'BA': 'BIH',
    'BB': 'BRB',
    'BD': 'BGD',
    'BE': 'BEL',
    'BF': 'BFA',
    'BG': 'BGR',
    'BH': 'BHR',
    'BI': 'BDI',
    'BJ': 'BEN',
    'BL': 'BLM',
    'BM': 'BMU',
    'BN': 'BRN',
    'BO': 'BOL',
    'BQ': 'BES',
    'BR': 'BRA',
    'BS': 'BHS',
    'BT': 'BTN',
    'BV': 'BVT',
    'BW': 'BWA',
    'BY': 'BLR',
    'BZ': 'BLZ',
    'CA': 'CAN',
    'CC': 'CCK',
    'CD': 'COD',
    'CF': 'CAF',
    'CG': 'COG',
    'CH': 'CHE',
    'CI': 'CIV',
    'CK': 'COK',
    'CL': 'CHL',
    'CM': 'CMR',
    'CN': 'CHN',
    'CO': 'COL',
    'CR': 'CRI',
    'CU': 'CUB',
    'CV': 'CPV',
    'CW': 'CUW',
    'CX': 'CXR',
    'CY': 'CYP',
    'CZ': 'CZE',
    'DE': 'DEU',
    'DJ': 'DJI',
    'DK': 'DNK',
    'DM': 'DMA',
    'DO': 'DOM',
    'DZ': 'DZA',
    'EC': 'ECU',
    'EE': 'EST',
    'EG': 'EGY',
    'EH': 'ESH',
    'ER': 'ERI',
    'ES': 'ESP',
    'ET': 'ETH',
    'FI': 'FIN',
    'FJ': 'FJI',
    'FM': 'FSM',
    'FO': 'FRO',
    'FR': 'FRA',
    'GA': 'GAB',
    'GB': 'GBR',
    'GD': 'GRD',
    'GE': 'GEO',
    'GF': 'GUF',
    'GG': 'GGY',
    'GH': 'GHA',
    'GI': 'GIB',
    'GL': 'GRL',
    'GM': 'GMB',
    'GN': 'GIN',
    'GP': 'GLP',
    'GQ': 'GNQ',
    'GR': 'GRC',
    'GT': 'GTM',
    'GU': 'GUM',
    'GW': 'GNB',
    'GY': 'GUY',
    'HK': 'HKG',
    'HM': 'HMD',
    'HN': 'HND',
    'HR': 'HRV',
    'HT': 'HTI',
    'HU': 'HUN',
    'ID': 'IDN',
    'IE': 'IRL',
    'IL': 'ISR',
    'IM': 'IMN',
    'IN': 'IND',
    'IO': 'IOT',
    'IQ': 'IRQ',
    'IR': 'IRN',
    'IS': 'ISL',
    'IT': 'ITA',
    'JE': 'JEY',
    'JM': 'JAM',
    'JO': 'JOR',
    'JP': 'JPN',
    'KE': 'KEN',
    'KG': 'KGZ',
    'KH': 'KHM',
    'KI': 'KIR',
    'KM': 'COM',
    'KN': 'KNA',
    'KP': 'PRK',
    'KR': 'KOR',
    'KW': 'KWT',
    'KY': 'CYM',
    'KZ': 'KAZ',
    'LA': 'LAO',
    'LB': 'LBN',
    'LC': 'LCA',
    'LI': 'LIE',
    'LK': 'LKA',
    'LR': 'LBR',
    'LS': 'LSO',
    'LT': 'LTU',
    'LU': 'LUX',
    'LV': 'LVA',
    'LY': 'LBY',
    'MA': 'MAR',
    'MC': 'MCO',
    'MD': 'MDA',
    'ME': 'MNE',
    'MF': 'MAF',
    'MG': 'MDG',
    'MH': 'MHL',
    'MK': 'MKD',
    'ML': 'MLI',
    'MM': 'MMR',
    'MN': 'MNG',
    'MO': 'MAC',
    'MP': 'MNP',
    'MQ': 'MTQ',
    'MR': 'MRT',
    'MS': 'MSR',
    'MT': 'MLT',
    'MU': 'MUS',
    'MV': 'MDV',
    'MW': 'MWI',
    'MX': 'MEX',
    'MY': 'MYS',
    'MZ': 'MOZ',
    'NA': 'NAM',
    'NC': 'NCL',
    'NE': 'NER',
    'NF': 'NFK',
    'NG': 'NGA',
    'NI': 'NIC',
    'NL': 'NLD',
    'NO': 'NOR',
    'NP': 'NPL',
    'NR': 'NRU',
    'NU': 'NIU',
    'NZ': 'NZL',
    'OM': 'OMN',
    'PA': 'PAN',
    'PE': 'PER',
    'PF': 'PYF',
    'PG': 'PNG',
    'PH': 'PHL',
    'PK': 'PAK',
    'PL': 'POL',
    'PM': 'SPM',
    'PN': 'PCN',
    'PR': 'PRI',
    'PT': 'PRT',
    'PW': 'PLW',
    'PY': 'PRY',
    'QA': 'QAT',
    'RE': 'REU',
    'RO': 'ROU',
    'RS': 'SRB',
    'RU': 'RUS',
    'RW': 'RWA',
    'SA': 'SAU',
    'SB': 'SLB',
    'SC': 'SYC',
    'SD': 'SDN',
    'SE': 'SWE',
    'SG': 'SGP',
    'SH': 'SHN',
    'SI': 'SVN',
    'SJ': 'SJM',
    'SK': 'SVK',
    'SL': 'SLE',
    'SM': 'SMR',
    'SN': 'SEN',
    'SO': 'SOM',
    'SR': 'SUR',
    'SS': 'SSD',
    'ST': 'STP',
    'SV': 'SLV',
    'SX': 'SXM',
    'SY': 'SYR',
    'SZ': 'SWZ',
    'TC': 'TCA',
    'TD': 'TCD',
    'TF': 'ATF',
    'TG': 'TGO',
    'TH': 'THA',
    'TJ': 'TJK',
    'TK': 'TKL',
    'TL': 'TLS',
    'TM': 'TKM',
    'TN': 'TUN',
    'TO': 'TON',
    'TR': 'TUR',
    'TT': 'TTO',
    'TV': 'TUV',
    'TW': 'TWN',
    'TZ': 'TZA',
    'UA': 'UKR',
    'UG': 'UGA',
    'UM': 'UMI',
    'US': 'USA',
    'UY': 'URY',
    'UZ': 'UZB',
    'VA': 'VAT',
    'VC': 'VCT',
    'VE': 'VEN',
    'VG': 'VGB',
    'VI': 'VIR',
    'VN': 'VNM',
    'VU': 'VUT',
    'WF': 'WLF',
    'WS': 'WSM',
    'YE': 'YEM',
    'YT': 'MYT',
    'ZA': 'ZAF',
    'ZM': 'ZMB',
    'ZW': 'ZWE'
}

In [5]:
country_name_to_iso3 = {
    'Afghanistan': 'AFG',
    'Åland Islands': 'ALA',
    'Albania': 'ALB',
    'Algeria': 'DZA',
    'American Samoa': 'ASM',
    'Andorra': 'AND',
    'Angola': 'AGO',
    'Anguilla': 'AIA',
    'Antarctica': 'ATA',
    'Antigua and Barbuda': 'ATG',
    'Argentina': 'ARG',
    'Armenia': 'ARM',
    'Aruba': 'ABW',
    'Australia': 'AUS',
    'Austria': 'AUT',
    'Azerbaijan': 'AZE',
    'Bahamas': 'BHS',
    'Bahrain': 'BHR',
    'Bangladesh': 'BGD',
    'Barbados': 'BRB',
    'Belarus': 'BLR',
    'Belgium': 'BEL',
    'Belize': 'BLZ',
    'Benin': 'BEN',
    'Bermuda': 'BMU',
    'Bhutan': 'BTN',
    'Bolivia': 'BOL',
    'Ivory Coast': 'CIV',
    'Bonaire, Sint Eustatius and Saba': 'BES',
    'Bosnia and Herzegovina': 'BIH',
    'Botswana': 'BWA',
    'Bouvet Island': 'BVT',
    'Brazil': 'BRA',
    'British Indian Ocean Territory': 'IOT',
    'Brunei Darussalam': 'BRN',
    'Bulgaria': 'BGR',
    'Burkina Faso': 'BFA',
    'Burundi': 'BDI',
    'Cabo Verde': 'CPV',
    'Cambodia': 'KHM',
    'Cameroon': 'CMR',
    'Canada': 'CAN',
    'Cayman Islands': 'CYM',
    'Central African Republic': 'CAF',
    'Chad': 'TCD',
    'Chile': 'CHL',
    'China': 'CHN',
    'Christmas Island': 'CXR',
    'Cocos (Keeling) Islands': 'CCK',
    'Colombia': 'COL',
    'Comoros': 'COM',
    'Republic of the Congo': 'COG',
    'Democratic Republic of the Congo': 'COD',
    'Cook Islands': 'COK',
    'Costa Rica': 'CRI',
    'Croatia': 'HRV',
    'Cuba': 'CUB',
    'Curaçao': 'CUW',
    'Cyprus': 'CYP',
    'Czechia': 'CZE',
    'Denmark': 'DNK',
    'Djibouti': 'DJI',
    'Dominica': 'DMA',
    'Dominican Republic': 'DOM',
    'Ecuador': 'ECU',
    'Egypt': 'EGY',
    'El Salvador': 'SLV',
    'Equatorial Guinea': 'GNQ',
    'Eritrea': 'ERI',
    'Estonia': 'EST',
    'Eswatini': 'SWZ',
    'Ethiopia': 'ETH',
    'Falkland Islands (Malvinas)': 'FLK',
    'Faroe Islands': 'FRO',
    'Fiji': 'FJI',
    'Finland': 'FIN',
    'France': 'FRA',
    'French Guiana': 'GUF',
    'French Polynesia': 'PYF',
    'French Southern Territories': 'ATF',
    'Gabon': 'GAB',
    'Gambia': 'GMB',
    'Georgia': 'GEO',
    'Germany': 'DEU',
    'Ghana': 'GHA',
    'Gibraltar': 'GIB',
    'Greece': 'GRC',
    'Greenland': 'GRL',
    'Grenada': 'GRD',
    'Guadeloupe': 'GLP',
    'Guam': 'GUM',
    'Guatemala': 'GTM',
    'Guernsey': 'GGY',
    'Guinea': 'GIN',
    'Guinea-Bissau': 'GNB',
    'Guyana': 'GUY',
    'Haiti': 'HTI',
    'Heard Island and McDonald Islands': 'HMD',
    'Holy See': 'VAT',
    'Honduras': 'HND',
    'Hong Kong': 'HKG',
    'Hungary': 'HUN',
    'Iceland': 'ISL',
    'India': 'IND',
    'Indonesia': 'IDN',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Ireland': 'IRL',
    'Isle of Man': 'IMN',
    'Israel': 'ISR',
    'Italy': 'ITA',
    'Jamaica': 'JAM',
    'Japan': 'JPN',
    'Jersey': 'JEY',
    'Jordan': 'JOR',
    'Kazakhstan': 'KAZ',
    'Kenya': 'KEN',
    'Kiribati': 'KIR',
    'Korea (Democratic People\'s Republic of)': 'PRK',
    'Korea (Republic of)': 'KOR',
    'Kuwait': 'KWT',
    'Kyrgyzstan': 'KGZ',
    'Lao People\'s Democratic Republic': 'LAO',
    'Latvia': 'LVA',
    'Lebanon': 'LBN',
    'Lesotho': 'LSO',
    'Liberia': 'LBR',
    'Libya': 'LBY',
    'Liechtenstein': 'LIE',
    'Lithuania': 'LTU',
    'Luxembourg': 'LUX',
    'Macao': 'MAC',
    'Madagascar': 'MDG',
    'Malawi': 'MWI',
    'Malaysia': 'MYS',
    'Maldives': 'MDV',
    'Mali': 'MLI',
    'Malta': 'MLT',
    'Marshall Islands': 'MHL',
    'Martinique': 'MTQ',
    'Mauritania': 'MRT',
    'Mauritius': 'MUS',
    'Mayotte': 'MYT',
    'Mexico': 'MEX',
    'Micronesia (Federated States of)': 'FSM',
    'Moldova (Republic of)': 'MDA',
    'Monaco': 'MCO',
    'Mongolia': 'MNG',
    'Montenegro': 'MNE',
    'Montserrat': 'MSR',
    'Morocco': 'MAR',
    'Mozambique': 'MOZ',
    'Myanmar': 'MMR',
    'Namibia': 'NAM',
    'Nauru': 'NRU',
    'Nepal': 'NPL',
    'Netherlands': 'NLD',
    'New Caledonia': 'NCL',
    'New Zealand': 'NZL',
    'Nicaragua': 'NIC',
    'Niger': 'NER',
    'Nigeria': 'NGA',
    'Niue': 'NIU',
    'Norfolk Island': 'NFK',
    'North Macedonia': 'MKD',
    'Northern Mariana Islands': 'MNP',
    'Norway': 'NOR',
    'Oman': 'OMN',
    'Pakistan': 'PAK',
    'Palau': 'PLW',
    'Palestine, State of': 'PSE',
    'Panama': 'PAN',
    'Papua New Guinea': 'PNG',
    'Paraguay': 'PRY',
    'Peru': 'PER',
    'Philippines': 'PHL',
    'Pitcairn': 'PCN',
    'Poland': 'POL',
    'Portugal': 'PRT',
    'Puerto Rico': 'PRI',
    'Qatar': 'QAT',
    'Réunion': 'REU',
    'Romania': 'ROU',
    'Russia': 'RUS',
    'Rwanda': 'RWA',
    'Saint Barthélemy': 'BLM',
    'Saint Helena, Ascension and Tristan da Cunha': 'SHN',
    'Saint Kitts and Nevis': 'KNA',
    'Saint Lucia': 'LCA',
    'Saint Martin (French part)': 'MAF',
    'Saint Pierre and Miquelon': 'SPM',
    'Saint Vincent and the Grenadines': 'VCT',
    'Samoa': 'WSM',
    'San Marino': 'SMR',
    'Sao Tome and Principe': 'STP',
    'Saudi Arabia': 'SAU',
    'Senegal': 'SEN',
    'Serbia': 'SRB',
    'Seychelles': 'SYC',
    'Sierra Leone': 'SLE',
    'Singapore': 'SGP',
    'Sint Maarten (Dutch part)': 'SXM',
    'Slovakia': 'SVK',
    'Slovenia': 'SVN',
    'Solomon Islands': 'SLB',
    'Somalia': 'SOM',
    'South Africa': 'ZAF',
    'South Georgia and the South Sandwich Islands': 'SGS',
    'South Sudan': 'SSD',
    'Spain': 'ESP',
    'Sri Lanka': 'LKA',
    'Sudan': 'SDN',
    'Suriname': 'SUR',
    'Svalbard and Jan Mayen': 'SJM',
    'Vietnam': 'VNM',
    'Sweden': 'SWE',
    'Switzerland': 'CHE',
    'Syrian Arab Republic': 'SYR',
    'Taiwan, Province of China': 'TWN',
    'Tajikistan': 'TJK',
    'Tanzania': 'TZA',
    'Thailand': 'THA',
    'Timor-Leste': 'TLS',
    'Togo': 'TGO',
    'Tokelau': 'TKL',
    'Tonga': 'TON',
    'Trinidad and Tobago': 'TTO',
    'Tunisia': 'TUN',
    'Turkey': 'TUR',
    'Turkmenistan': 'TKM',
    'Turks and Caicos Islands': 'TCA',
    'Tuvalu': 'TUV',
    'Uganda': 'UGA',
    'Ukraine': 'UKR',
    'United Arab Emirates': 'ARE',
    'United Kingdom': 'GBR',
    'United States': 'USA',
    'Uruguay': 'URY',
    'Uzbekistan': 'UZB',
    'Vanuatu': 'VUT',
    'Venezuela': 'VEN',
    'Viet Nam': 'VNM',
    'Wallis and Futuna': 'WLF',
    'Western Sahara': 'ESH',
    'Yemen': 'YEM',
    'Zambia': 'ZMB',
    'Zimbabwe': 'ZWE',
    'South Korea' : 'KOR',
    'North Korea' : 'PRK'
}

In [6]:
display(AS_Country_Vendor_df.dropna())

,country_code,country,vendor,continent
107,NG,Nigeria,Cisco,Africa
108,BG,Bulgaria,Cisco,Europe
109,AE,United Arab Emirates,Cisco,Asia
110,TH,Thailand,Cisco,Asia
111,TH,Thailand,Cisco,Asia
...,...,...,...,...
1495691,EC,Ecuador,MikroTik,South America
1495692,US,United States,MikroTik,North America
1495693,HN,Honduras,MikroTik,North America
1495694,MX,Mexico,MikroTik,North America


In [7]:
display(Vendor_Version_Country_df.dropna())

,vendor,version,country
107,Cisco,15.5(3)S2,Nigeria
109,Cisco,16.9.4,United Arab Emirates
110,Cisco,16.7.1,Thailand
111,Cisco,15.5(3)S2,Thailand
112,Cisco,12.2(53)SE,Argentina
...,...,...,...
1495580,MikroTik,6.49.6,Russia
1495622,SonicWall,6.5.4.12-101n,United States
1495629,MikroTik,6.48.5,Yemen
1495658,Tandberg,TC7.3.2.14ad7cc,Indonesia


In [8]:
display(Vendor_Country_df.dropna())

,vendor,country,continent
107,Cisco,Nigeria,Africa
108,Cisco,Bulgaria,Europe
109,Cisco,United Arab Emirates,Asia
110,Cisco,Thailand,Asia
111,Cisco,Thailand,Asia
...,...,...,...
1495691,MikroTik,Ecuador,South America
1495692,MikroTik,United States,North America
1495693,MikroTik,Honduras,North America
1495694,MikroTik,Mexico,North America


In [9]:
data = AS_Country_Vendor_df.dropna()

continent_vendor_counts = data.groupby(['continent', 'vendor']).size().reset_index(name='continent_count')
continent_vendor_counts = continent_vendor_counts.sort_values(by='continent_count', ascending=False)
continent_vendor_counts = continent_vendor_counts[continent_vendor_counts['continent_count'] >= 2000]
fig = px.bar(continent_vendor_counts, x='continent', y='continent_count', color='vendor', title='Number of Vendors On Different Continents')

fig.update_layout(
    xaxis_title="Continent",   
    yaxis_title="Number of Vendors",       
)

fig.update_layout(
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    ),
    legend=dict(
        orientation='h',  
        x=0.30, y=1.2,  
        bgcolor='rgba(255, 255, 255, 0.5)',  
        bordercolor='rgba(0, 0, 0, 0.5)',  
        borderwidth=1,  
        title=''
    )
)

fig.show()

In [10]:
dataframe_used = AS_Country_Vendor_df.dropna()

dataframe_used.loc[:, 'vendor'] = dataframe_used['vendor'].apply(lambda x: x.lower())

column_to_count = 'vendor'

column_to_create = 'Vendor_count'

column_to_count2 = 'country_code'

column_to_create2 = 'CountryCode_count'

vendor_counts = dataframe_used.groupby(['country_code', 'vendor']).size().reset_index(name='count')
vendor_counts = vendor_counts[vendor_counts['count'] >= 2000]
vendor_counts = vendor_counts.sort_values(by='count', ascending=False)

fig = px.bar(vendor_counts, x='country_code', y='count', color='vendor', title='Number of Vendors in Different ASes')

fig.update_layout(
    xaxis_title="AS Country Code",   
    yaxis_title="Number of Vendors",      
)

fig.update_layout(
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    ),
    legend=dict(
        orientation='h',  
        x=0.35, y=1.2,  
        bgcolor='rgba(255, 255, 255, 0.5)',  
        bordercolor='rgba(0, 0, 0, 0.5)',  
        borderwidth=1,  
        title=''
    )
)

fig.show()

In [11]:
data = AS_Country_Vendor_df.dropna().copy()
data.loc[:, 'vendor'] = data['vendor'].apply(lambda x: x.lower())
data['country_code'] = data['country_code'].map(country_code_mapping)
most_common_vendors = data.groupby('country_code')['vendor'].agg(lambda x: x.value_counts().index[0]).reset_index()

fig = px.choropleth(
    most_common_vendors,
    locations='country_code',
    locationmode='ISO-3',
    color='vendor',
    hover_name='vendor',
    title='Most Common Vendor per AS',
    projection='natural earth', 
    scope='world'  
)

fig.update_geos(
    showcountries=True, 
    countrycolor="Black"
)

fig.update_layout(
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    )
)


fig.update_layout(
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    ),
    legend=dict(
        orientation='h',  
        x=0.28, y=1.2, 
        bgcolor='rgba(255, 255, 255, 0.5)',  
        bordercolor='rgba(0, 0, 0, 0.5)', 
        borderwidth=1, 
        title=''
    )
)

fig.show()

In [12]:
data = AS_Country_Vendor_df.dropna().copy()

data.loc[:, 'iso_alpha3'] = data['country_code'].map(country_code_mapping)

vendor_counts = data.groupby('iso_alpha3')['vendor'].size().reset_index(name='VendorCount')

world_map = px.choropleth(
    vendor_counts,
    locations='iso_alpha3',
    locationmode='ISO-3',
    color='VendorCount',
    hover_name='iso_alpha3',
    color_continuous_scale='Viridis',
    range_color=(0, vendor_counts['VendorCount'].max()),  
    labels={'VendorCount': 'Number of SNMP-enabled devices'},
    title='Total Number of SNMP-enabled devices per AS',
    projection='natural earth',  
    scope='world'  
)


world_map.update_layout(
    legend=dict(
        orientation='h',  
        yanchor='top',  
        y=1.02,  #
        xanchor='center',  
        x=10,  
        bgcolor='rgba(255, 255, 255, 0.5)', 
        bordercolor='rgba(0, 0, 0, 0.5)',  
        borderwidth=1,  
        title='' 
    ),
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    )
)

world_map.show()

In [13]:
dataframe_used = AS_Country_Vendor_df.dropna()

dataframe_used.loc[:, 'country'] = dataframe_used['country'].apply(lambda x: x.lower())

column_to_count = 'vendor'

column_to_create = 'Vendor_count'

column_to_count2 = 'country'

column_to_create2 = 'Country_count'

vendor_counts = dataframe_used.groupby(['country', 'vendor']).size().reset_index(name='count')
vendor_counts = vendor_counts[vendor_counts['count'] >= 2000]
vendor_counts = vendor_counts.sort_values(by='count', ascending=False)

fig = px.bar(vendor_counts, x='country', y='count', color='vendor', title='Number of Vendors in Different Countries')

fig.update_layout(
    xaxis_title="Country", 
    yaxis_title="Number of Vendors",      
)

fig.update_layout(
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    ),
    legend=dict(
        orientation='h',  
        x=0.38, y=1.2,  
        bgcolor='rgba(255, 255, 255, 0.5)', 
        bordercolor='rgba(0, 0, 0, 0.5)', 
        borderwidth=1, 
        title=''
    )
)

fig.show()

In [14]:
data = AS_Country_Vendor_df.dropna().copy()
data['country'] = data['country'].map(country_name_to_iso3)
most_common_vendors = data.groupby('country')['vendor'].agg(lambda x: x.value_counts().index[0]).reset_index()

fig = px.choropleth(
    most_common_vendors,
    locations='country',
    locationmode='ISO-3',
    color='vendor',
    hover_name='vendor',
    title='Most Common Vendor per Country',
    projection='natural earth',  
    scope='world' 
)

fig.update_geos(
    showcountries=True, 
    countrycolor="Black"
)

fig.update_layout(
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    )
)

fig.update_layout(
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    ),
    legend=dict(
        orientation='h',  
        x=0.25, y=1.2,  
        bgcolor='rgba(255, 255, 255, 0.5)', 
        bordercolor='rgba(0, 0, 0, 0.5)',  
        borderwidth=1,  
        title=''
    )
)


fig.show()

In [15]:
data = AS_Country_Vendor_df.dropna().copy()

data.loc[:, 'iso_alpha3'] = data['country'].map(country_name_to_iso3)

vendor_counts = data.groupby('iso_alpha3')['vendor'].size().reset_index(name='VendorCount')

world_map = px.choropleth(
    vendor_counts,
    locations='iso_alpha3',
    locationmode='ISO-3',
    color='VendorCount',
    hover_name='iso_alpha3',
    color_continuous_scale='Viridis',
    range_color=(0, vendor_counts['VendorCount'].max()),  
    labels={'VendorCount': 'Number of SNMP-enabled devices'},
    title='Total Number of SNMP-enabled devices per Country',
    projection='natural earth',  
    scope='world' 
)

world_map.update_layout(
    geo=dict(
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
        resolution=50
    )
)

world_map.show()